In [1]:
from collections import defaultdict
import re

# ---- TRAINING ----

def preprocess(text):
    # Lowercase + simple tokenization (split by non-alphanum)
    return re.findall(r'\w+', text.lower())

def build_inverted_index(docs):
    """
    docs: list of strings (e.g., ["The cat sat", "The dog barked"])
    returns: inverted index (word -> set of doc ids)
    """
    index = defaultdict(set)
    for doc_id, text in enumerate(docs):
        tokens = preprocess(text)
        for token in set(tokens):  # use set to avoid duplicate entries
            index[token].add(doc_id)
    return index

# ---- INFERENCE ----

def search(index, query):
    """
    query: string, e.g., "cat"
    returns: list of matching document IDs
    """
    tokens = preprocess(query)
    if not tokens:
        return []
    
    # Start with the first token's postings
    result = index.get(tokens[0], set()).copy()
    
    # Intersect with other token postings (AND query)
    for token in tokens[1:]:
        result &= index.get(token, set())
    
    return sorted(result)

# ---- EXAMPLE USAGE ----

# Sample document collection
documents = [
    "The cat sat on the mat.",
    "The dog barked loudly.",
    "A cat and a dog became friends.",
    "The quick brown fox."
]

# Train
inv_index = build_inverted_index(documents)

# Inference
queries = ["cat", "dog", "cat dog", "fox", "unicorn"]

for q in queries:
    result = search(inv_index, q)
    print(f"Query: '{q}' → Docs: {result}")

Query: 'cat' → Docs: [0, 2]
Query: 'dog' → Docs: [1, 2]
Query: 'cat dog' → Docs: [2]
Query: 'fox' → Docs: [3]
Query: 'unicorn' → Docs: []
